In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


#arquivo de dados
df = pd.read_csv("../data/dataset.csv")
casas_sem_preco = df[df['Price'].isnull()].copy()


# renomear as colunas para português
df.rename(columns={
    'Suburb': 'Subúrbio',
    'Address': 'Endereço',
    'Rooms': 'Salas',
    'Type': 'Tipo',
    'Price': 'Preço',
    'Method': 'Método de Venda',
    'SellerG': 'Vendedor',
    'Date': 'Data',
    'Distance': 'Distância',
    'Postcode': 'Código Postal',
    'Bedroom2': 'Quartos',
    'Bathroom': 'Banheiros',
    'Car': 'Garagem',
    'Landsize': 'Tamanho do Terreno',
    'BuildingArea': 'Área Construída',
    'YearBuilt': 'Ano de Construção',
    'CouncilArea': 'Área Administrativa',
    'Lattitude': 'Latitude',
    'Longtitude': 'Longitude',
    'Regionname': 'Nome da Região',
    'Propertycount': 'Quantidade de Imóveis na Região'
}, inplace=True)

#display(df.head())

In [ ]:
def dados_estatisticos(df):
    print("\nFormato do DataFrame (linhas, colunas):", df.shape)
    print("\nNúmero total de linhas completas:", df.dropna().shape[0])
    print("\nColunas com valores ausentes (ordem decrescente):")
    print(df.isnull().sum().sort_values(ascending=False))

    print("\nEstatísticas descritivas para colunas numéricas:")
    print(df.describe())


    print("\nFrequência relativa das colunas categóricas :")
    for col in df.select_dtypes(include='object').columns:
        print(f"\n{col}:")
        print(df[col].value_counts(normalize=True).head(10))
    
    if 'Preço' in df.columns:
        print("\nCorrelação com a coluna 'Preço':")
        print(df.corr(numeric_only=True)['Preço'].sort_values(ascending=False))

        print("\n Histograma da variável 'Preço':")
        df['Preço'].hist(bins=50)
        plt.xlabel("Preço")
        plt.ylabel("Frequência")
        plt.title("Distribuição de Preços")
        plt.grid(True)
        plt.show()
        

    print("\n Total de linhas duplicadas:", df.duplicated().sum())
    

display(dados_estatisticos(df))


# Contar quantos NaN existem por bairro
df_nan_garagem = df[df['Garagem'].isnull()]
nan_garagem_por_bairro = df_nan_garagem.groupby('Subúrbio').size().sort_values(ascending=False)
print(nan_garagem_por_bairro)


In [ ]:
# como visto anteriormente, dados ausentes podem afetar a análise, então é importante tratá-los
# Área Construída, Ano de Construção e Área Administrativa são colunas com muitos valores ausentes
df.drop(columns=['Área Construída', 'Ano de Construção', 'Área Administrativa', 'Vendedor'], inplace=True)
df = df.dropna(subset=['Preço'])


#Colunas como garagem, banheiros e quartos podem ser preenchidos com a mediana, separados por bairro 
def preencher_por_bairro(df, col):
    # mediana da coluna por bairro
    mediana_por_bairro = df.groupby('Subúrbio')[col].median()

    # Função auxiliar que preenche com a mediana do bairro ou com a mediana global
    def preencher(row):
        if pd.isna(row[col]):
            bairro = row['Subúrbio']
            # se o bairro não está no índice ou a mediana é NaN, usa a mediana global PODE ALTERAR RESULTADOS
            if bairro in mediana_por_bairro and not pd.isna(mediana_por_bairro[bairro]):
                return mediana_por_bairro[bairro]
            else:
                return df[col].median()
        else:
            return row[col]

    df[col] = df.apply(preencher, axis=1)
    return df

preencher_por_bairro(df, 'Garagem')
preencher_por_bairro(df, 'Banheiros')
preencher_por_bairro(df, 'Quartos')
df['Subúrbio'].value_counts().head(60)


display(df)